In [37]:
import glob, os
from tqdm import tqdm
# os.chdir("./quiztoolbox.com/")
confirmed_questions = set()
all_questions = set()
question_files = []
page_titles = []
sitename = "coursecopy"
for file in glob.glob("./" + sitename + ".com/**/*.html", recursive=True):
    with open(file, 'r') as file_reader:
        try:
            file_contents = file_reader.read()
#             print(file_contents)
            if "show answer" in file_contents.lower():
#                 print(file)
                question_files.append(file)
        except UnicodeDecodeError:
            pass
#             print("skipping", file)
# display(question_files)
print(len(question_files))

255


In [38]:
class question():
    def __init__(self, text, tag=None, site=''):
        self.text = text
        if tag:
            self.tag = tag
        else:
            self.tag = ''
        self.site = site
        
    def __lt__(self, other):
        return self.text < other.text
    
    def __eq__(self, other):
        return self.text == other.text and self.tag == other.tag
    
    def __hash__(self):
        return self.text.__hash__() + (self.tag.__hash__() if self.tag else 0)
    
    def __repr__(self):
        return str({'text': self.text, 'site': self.site, 'tag:': self.tag})

In [39]:
from bs4 import BeautifulSoup
from bs4 import UnicodeDammit
import html.parser as hp
from collections import Counter
import unidecode

In [40]:
def get_duplictes(lst):
    # https://stackoverflow.com/questions/52072381/how-to-print-only-the-duplicate-elements-in-python-list
    d =  Counter(lst)  # -> Counter({4: 3, 6: 2, 3: 1, 2: 1, 5: 1, 7: 1, 8: 1})
    res = [k for k, v in d.items() if v > 1]
    return res

In [41]:
for file in tqdm(question_files):
#     file = question_files[-3]
#     print(file)
    with open(file, 'r') as file_reader:
    #     file_contents = UnicodeDammit(file_reader.read()).unicode_markup
        file_contents = hp.unescape(file_reader.read())

    parser = BeautifulSoup(file_contents, "lxml")
    # print(parser)
    tag = parser.find("meta",  property="og:description")
    if tag:
        tag = tag['content']
    headings = parser.find_all("div", class_="panel-heading")
    questions = []
    for h in headings:
        questions.append(question(unidecode.unidecode(h.text).strip(), tag, sitename))
    #     questions.append(h.get_text())

    questions = sorted(questions)
    all_questions.update(questions)
    found = get_duplictes(questions)
#     if len(found)==0:
#         print("missing questions on ", file)
    confirmed_questions.update(found)

100%|██████████| 255/255 [00:03<00:00, 76.78it/s]


In [42]:
print(len(confirmed_questions))
# display(sorted(confirmed_questions))
display(sorted(confirmed_questions, key=lambda q:q.tag))

139


[{'text': 'The beautiful red H-alpha spectral line of hydrogen is actually', 'site': 'coursecopy', 'tag:': ''},
 {'text': 'In the kidney proximal tubules, which vitamin binds to a cubulin-megalin membrane receptor to form a complex and be activated?', 'site': 'coursecopy', 'tag:': ''},
 {'text': 'The image below illustrates a set of deciduous dentition. What is the type and location of the tooth labelled M?      [answer]', 'site': 'coursecopy', 'tag:': ''},
 {'text': 'Where do you store glycogen (to be broken down as a supply of glucose)?', 'site': 'coursecopy', 'tag:': ''},
 {'text': 'Shelly Watters owns a chain of fashion boutiques that started in Washington, DC and has expanded into Maryland, Virginia, and West Virginia. Shelly is growing her company via a strategy of ________.', 'site': 'coursecopy', 'tag:': ''},
 {'text': 'In the experiments constructed by Pasteur to disprove spontaneous generation, swan-necked flasks were used. Why was this shape of flask used in this experiment?

In [43]:
fake_questions = all_questions.difference(confirmed_questions)

In [44]:
print(len(all_questions))
print(len(confirmed_questions))

2266
139


In [45]:
for s in confirmed_questions:
    if "honorlock".lower() in s.text.lower():
        print(s)

{'text': 'You should make sure you keep the HonorLock shared screen and webcam on throughout the entire exam duration.', 'site': 'coursecopy', 'tag:': ''}


In [2]:
import pickle

In [72]:
file = open(sitename + '.all_q.txt', 'wb')
pickle.dump(all_questions, file)
file.close()

In [73]:
file = open(sitename + '.confirmed_q.txt', 'wb')
pickle.dump(confirmed_questions, file)
file.close()

In [74]:
if os.path.exists("all_questions"):
    with open("all_questions","rb") as all_q:
        all_q_data = set(pickle.load(all_q))
    with open("all_questions","wb") as all_q:
        all_q_data.update(confirmed_questions)
        pickle.dump(all_q_data, all_q)
else:
    file = open("all_questions", 'wb')
    pickle.dump(confirmed_questions, file)
    file.close()

In [11]:
all_q = open("all_questions","rb")
all_q_data = set(pickle.load(all_q))
questions_master = sorted(all_q_data)
print(len(questions_master))
for q in questions_master:
#     if("study" in q.site):
#         print(q)
    if("demo" in q.text.lower()):
        print(q.text)
all_q.close()

1019
Demo Question 1
Honorlock's demo contains a ______ in a shield.
Product X contribution margin minus direct product fixed expenses is termed?   https://demo.honorlock.com/dmca/903/results
Which one of the following statements correctly calls a method named ShowName? https://demo.honorlock.com/dmca/903/results   Search & Destroy
While being recorded, please demonstrate your knowledge of removing and adding the Honorlock LTI to a course. First, please make sure you are in the course titled "HL Training - Sandbox_test".Step 1: Locate the Honorlock LTI and delete it. Step 2: Add the Honorlock LTI to the course once more.Did you manage to complete this task?


In [47]:
with open("paste_out.txt", "w") as writer:
    for c in confirmed_questions:
        writer.write(c.text+"\n\n")